In [1]:
import argparse
import cv2
import os
import numpy as np
from tqdm import tqdm
import re
import imutils

In [2]:
def write_next_img(filename, txtname):
    f = open(txtname, "a")
    f.write("\n"+filename+" ")
    f.close()
    
def write_rect(rect):
    global TXT
    f = open(TXT, "a")
    f.write(np.array_str(rect)+" ")
    f.close()

def click_and_crop(event, x, y, flags, param):
    global rect, tip_x, tip_y, clone
    h,w,_ = clone.shape
    if event == cv2.EVENT_LBUTTONDOWN:
        rect=np.concatenate((rect,[x,y]))
        if len(rect)==4:
            write_rect(rect)
            rect=np.array([])
    elif event == cv2.EVENT_MOUSEMOVE:
        white=(255,255,255)
        clone  = image.copy()
        if len(rect)==2:
            cv2.rectangle(clone, tuple(map(int,rect)), (x,y), (0,255,0))
        cv2.line(clone, (0,y),(w,y),white)
        cv2.line(clone, (x,0),(x,h),white)
        cv2.imshow("image", clone)

# Set paths

In [3]:
SCENE = "stack_1"
NEW_RESOLUTION=None#(1040,780)

DIR = "/home/maciej/Desktop/pallet_dataset/"+SCENE+"/"
TXT = "/home/maciej/Desktop/pallet_dataset/labels/"+SCENE+".txt"

# Label

In [ ]:
filenames = sorted(os.listdir(DIR))
#filenames=["0140.jpg"]
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

#variables
rect = np.array([])
tip_x, tip_y = (0,0)

for i in tqdm(range(len(filenames))):
    filename=filenames[i]
    path=DIR+filename
    image = cv2.imread(path)
    image = imutils.rotate(image, 1.5)
    if NEW_RESOLUTION:
        image = cv2.resize(image, NEW_RESOLUTION)
    clone = image.copy()
    write_next_img(filename, TXT)
    cv2.imshow("image", clone)
    cv2.waitKey(0)
    rect = np.array([])
cv2.destroyAllWindows()

 11%|█         | 5/46 [02:30<21:13, 31.06s/it]

# Display labelled

In [ ]:
def parse_line(line):
    filename,points=line.split(" ", 1)
    point_list=points.split("] [")
    point_list=[points.strip('][ ').replace('.','').split(' ') for points in point_list]
    point_list=[[int(point) for point in points if point] for points in point_list]
    point_list=[(tuple(points[:2]),tuple(points[2:4])) for points in point_list]
    point_list=[((0,0),(0,0)) if not points[0] and not points[1] else points for points in point_list]
    print(point_list)
    return filename,point_list

with open(TXT) as f:
    lines = [line.rstrip('\n') for line in f]
lines = [parse_line(line) for line in lines]
cv2.namedWindow("image")
for line in lines:
    filename,point_list=line
    path=DIR+filename
    image = cv2.imread(path)
    if NEW_RESOLUTION:
        image = cv2.resize(image, NEW_RESOLUTION) 
    for points in point_list:
        p1,p2=points
        cv2.rectangle(image, p1, p2, (0,255,0))
    cv2.imshow("image", image)
    cv2.waitKey(0)
cv2.destroyAllWindows()